In [1]:
import os
current_directory = os.getcwd()
%run packages.py
 #Download all required packages in one fell swoop. 

In [2]:
%load_ext cython

In [3]:
%%cython --annotate
cimport numpy as np

def getAuth():
    dotenv_path = '/Users/mariam/Documents/GitHub/advanced_python/credentials.env' # Put your credentials in.
    load_dotenv(dotenv_path=dotenv_path)  
    
    client_id, client_secret = os.environ.get('client_id'), os.environ.get('client_secret') 
    credentials = f"{client_id}:{client_secret}"
    credentials_b64 = base64.b64encode(credentials.encode())
    
    url = 'https://accounts.spotify.com/api/token'
    token_data = {'grant_type': 'client_credentials'}
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'Basic {credentials_b64.decode()}'
    }
    response = requests.post(url, data=token_data, headers=headers)
    if response.ok:
        return response.json().get('access_token')
    else:
        print(f"Invalid: {response.status_code}, {response.text}")
        return None

#Pull all user playlists for processing, specifically returning a dict of playlist ids & titles. 
def getPlaylistsid(access_token):
    user_id = os.environ.get("user_id")
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(f"https://api.spotify.com/v1/users/{user_id}/playlists", headers=headers)

    if response.ok:
        playlists_json = response.json()
        playlist_info = [
            {"id": playlist["id"], "name": playlist["name"]}
            for playlist in playlists_json.get("items", [])
        ]
        return playlist_info
    else:
        print(f"Invalid: {response.status_code}, {response.text}")
        return None

# Pulls all tracks from a list of playlists and returns a DataFrame with relevant track details. 
def getNumpyplaylist(playlist_info, access_token):
    
    cdef np.ndarray df_tracks

    headers = {"Authorization": f"Bearer {access_token}"}

    def fetch_playlist(playlist_id):
        all_tracks = []
        url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
        while url:
            response = requests.get(url, headers=headers, params={"limit": 100})
            if response.ok:
                data = response.json()
                all_tracks.extend(data.get("items", []))
                url = data.get("next")
            else:
                print(f"Invalid {playlist_id}: {response.status_code}, {response.text}")
                return []
        return all_tracks

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        results = executor.map(fetch_playlist, [info["id"] for info in playlist_info])

    all_tracks = list(itertools.chain.from_iterable(results))

    playlist_id_name_map = {info["id"]: info["name"] for info in playlist_info}

    tracks = []
    for item in all_tracks:
        track = item.get("track")
        if track:
            playlist_id = item.get("playlist_id")
            playlist_name = playlist_id_name_map.get(playlist_id, "Unknown")
            
            track_info = {
                "id": track.get("id"),
                "title": track.get("name"),
                "popularity": track.get("popularity"),
                "album": track["album"].get("name"),
                "release_date": track["album"].get("release_date"),
                "album_total_tracks": track["album"].get("total_tracks"),
                "artist": ', '.join([artist["name"] for artist in track["artists"]]), 
                "artist_id": track["artists"][0]["id"] if track["artists"] else None,  
                "album_id": track["album"].get("id"),
                "playlist_name": playlist_name
            }
            tracks.append(track_info)

    df_tracks = np.array([list(map(lambda x: x[key], tracks)) for key in tracks[0].keys()])
    return df_tracks

#Pulls relevant artists details. 
def getNumpyartist(artist_ids, access_token):
    cdef int MAX_IDS = 50
    cdef np.ndarray df_artists

    artists_info = []
    headers = {"Authorization": f"Bearer {access_token}"}
    
    # Batch requests to optimize network usage
    for i in range(0, len(artist_ids), MAX_IDS):
        chunk = artist_ids[i:i + MAX_IDS]
        artist_ids_str = ",".join(chunk)
        
        response = requests.get(f"https://api.spotify.com/v1/artists?ids={artist_ids_str}", headers=headers)
        
        if response.ok:
            chunk_data = response.json()
            artists_info.extend(chunk_data.get("artists", []))
        else:
            print(f"Invalid: {response.status_code}, {response.text}")
    
    artists = []
    for artist in artists_info:
        artist_info = {
            "followers": artist.get("followers", {}).get("total"),
            "genres": "_".join(artist.get("genres")),
            "id": artist.get("id"),
            "name": artist.get("name"),
            "popularity": artist.get("popularity"),
        }
        artists.append(artist_info)
    
    df_artists = np.array([list(map(lambda x: x[key], artists)) for key in artists[0].keys()])
    return df_artists

#Notes:above ^ dedupe artist prior reducing redundancy and unnecessary data processing.

# Pulls comprehensive audio details for input tracks.
def getnumpyAudiofeatures(track_ids, access_token):

    cdef int MAX_TRACKS = 100
    cdef np.ndarray[np.float64_t, ndim=2] df_features

    all_features = []  
    
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    for i in range(0, len(track_ids), MAX_TRACKS):
        chunk = track_ids[i:i+MAX_TRACKS]
        track_ids_str = ",".join(chunk)

        response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={track_ids_str}", headers=headers)
        if response.ok:
            data = response.json()
            all_features.extend(data.get('audio_features', []))
        else:
            print(f"Invalid: {response.status_code}, {response.text}")
            return None  

    features_data = []
    for feature in all_features:
        if feature:  
            features = {
                'danceability': feature.get('danceability'),
                'energy': feature.get('energy'),
                'key': feature.get('key'),
                'loudness': feature.get('loudness'),
                'mode': feature.get('mode'),
                'speechiness': feature.get('speechiness'),
                'acousticness': feature.get('acousticness'),
                'instrumentalness': feature.get('instrumentalness'),
                'liveness': feature.get('liveness'),
                'valence': feature.get('valence'),
                'tempo': feature.get('tempo'),
                'duration_ms': feature.get('duration_ms'),
                'time_signature': feature.get('time_signature'),
                #'id': feature.get('id')
            }
            features_data.append(features)
    
    df_features = np.array([list(map(lambda x: x[key], features_data)) for key in features_data[0].keys()])
    return df_features

def plotTracks(df_tracks):

    #filter out poorly formatted dates
    mylen = np.vectorize(len)
    release_date = np.sort([np.datetime64(i, 'Y') for i in df_tracks[4][mylen(df_tracks[4])==10]]).astype(str)
    plt.figure(figsize=(10, 6))
    num, bins, _ = plt.hist(release_date, color='purple', edgecolor='black', rwidth=0.7, align='mid')
    plt.xlabel('Year of Release')
    plt.ylabel('Number of Songs')
    plt.title('Songs by Year of Release')
    plt.xticks(rotation=45)
    plt.show()

def plotArtists(df_artists):
    
    #changed word cloud not to link to df_tracks, couldn't make it work..
    all_genres = ' '.join([genre for sublist in [i.split('_') for i in df_artists[1]] if sublist!= [] for genre in sublist])
    wordcloud = WordCloud(width = 800, height = 800,
                          background_color ='white', 
                          colormap='inferno',  
                          min_font_size = 10).generate(all_genres)
    
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    
    plt.show()

    all_genres = [genre for sublist in [i.split('_') for i in df_artists[1]] if isinstance(sublist, list) for genre in sublist]
    genre_counts = pd.Series(all_genres).value_counts().nlargest(10)  #Get the top 10 genres.
    
    data = {
        'genres': genre_counts.index.tolist(),
        'counts': genre_counts.values
    }
    source = ColumnDataSource(data=data)
    
    p = figure(x_range=data['genres'], title="Top 10 Genres Based on Primary Genre of Artists",
               toolbar_location=None, tools="", height=400)
    
    hover = HoverTool()
    hover.tooltips = [
        ("Genre", "@genres"),
        ("Count", "@counts")
    ]
    p.add_tools(hover)
    
    p.vbar(x='genres', top='counts', width=0.9, source=source, fill_color='purple', line_color=None)
    
    p.xaxis.major_label_orientation = 1.2
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    output_notebook()
    show(p)

    bin_edges = np.arange(0, 105, 5)
    counts, _ = np.histogram(df_artists[4].astype(int), bins=bin_edges)
    percentages = (counts / len(df_artists[4])) * 100
    avg_popularity = df_artists[4].astype(int).mean()

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(range(len(bin_edges)-1), percentages, color='purple', edgecolor='black')
    for bar, count in zip(bars, counts):
        if count !=0:
            ax.annotate(f'{count}', 
                        xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                        xytext=(0, 3),  
                        textcoords="offset points",
                        ha='center', va='bottom')

    ax.set_xticks(range(len(bin_edges)-1))
    ax.set_xticklabels([f'{bin_edges[i]}-{bin_edges[i+1]}' for i in range(len(bin_edges)-1)], rotation=45, ha="right")

    ax.set_xlabel('Popularity Score Range')
    ax.set_ylabel('Percentage of Artists')
    ax.set_title('Overall Playlist Popularity')

    caption = f"How unique is your taste? This playlist's average popularity score is roughly {avg_popularity:.1f}"
    fig.text(0.5, 0.01, caption, ha='center', va='bottom', fontsize=10, color='gray')

    plt.tight_layout()
    plt.show()

    all_genres = set()
    for genres in [i.split('_') for i in df_artists[1]]:
        all_genres.update(genres)
    total_unique_genres = len(all_genres)
    
    # Prep data for the top 10 artists by genre count.
    genre_count = np.array([len(j) for j in [i.split('_') for i in df_artists[1]]])
    top_artists = df_artists[3][genre_count.argsort()[-10:][::-1]]
    genre_counts = genre_count[genre_count.argsort()[-10:][::-1]]
    top_genres = [i.split('_') for i in df_artists[1][genre_count.argsort()[-10:][::-1]]]

    source = ColumnDataSource(data=dict(
        names=top_artists,
        counts=genre_count,
        genres=['; '.join(genres) for genres in top_genres]
    ))
    
    p = figure(x_range=top_artists, height=400, title="Top 10 Artists by Genre Diversity",
               toolbar_location=None, tools="")
    p.vbar(x='names', top='counts', width=0.9, source=source, legend_field="names", 
           line_color='white', fill_color='purple')
    
    p.add_tools(HoverTool(tooltips=[("Artist", "@names"), ("# Genres", "@counts"), ("Genres", "@genres")]))
    
    p.xaxis.major_label_orientation = "vertical"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    caption_text = f'<div><strong>Total Unique Genres in Playlist: {total_unique_genres}</strong></div>'
    caption = Div(text=caption_text)
    layout = column(p, caption)
    
    output_notebook() 
    show(layout)

def plotAudioFeatures(df_audio_features):
    features = ['danceability', 'energy', 'liveness']
    feature_id = [0, 1, 8]
    data = [df_audio_features[feature] for feature in feature_id]
    
    colors = ['skyblue', 'lightgreen', 'salmon']
    
    fig, ax1 = plt.subplots(figsize=(14, 6))
    # plt.style.use('seaborn-deep')

    ax1.hist(data, bins=20, label=features)
    ax1.set_xlabel("Scale")
    ax1.set_ylabel("Frequency")
    ax1.legend(loc='upper left')
    
    ax2 = ax1.twinx()
    
    box = ax2.boxplot(data, vert=False, positions=np.arange(len(data)) + 1, patch_artist=True)
    
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
    
    ax2.yaxis.tick_right()
    ax2.set_yticks(np.arange(1, len(data) + 1))
    ax2.set_yticklabels(features)
    ax2.yaxis.set_label_position("right")
    ax2.set_ylabel("Features")

    plt.title("Audio Features Distributions")
    plt.show()

    plt.hist(df_audio_features[11], color='purple', rwidth=0.7)
    plt.xlabel("Song Duration")
    plt.xticks(rotation=90)
    plt.ylabel("Frequency")
    plt.title("Song Duration Distribution")
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.hist(df_audio_features[10], bins=30, color='purple', edgecolor='black')
    plt.xlabel('Tempo (BPM)')
    plt.ylabel('Number of Songs')
    plt.title('Frequency of Song Tempos')
    plt.show()

def plotTopofPlaylist(df_tracks, artists_df):
    fig, axs = plt.subplots(1, 2, figsize=(20, 6)) 

    top_5_songs = df_tracks[2].astype(int).argsort()[-5:][::-1]
    axs[0].bar(df_tracks[1][top_5_songs], df_tracks[2][top_5_songs].astype(int), color='purple', width=0.5)
    axs[0].set_title('Top 5 Most Popular Songs in Playlist')
    axs[0].set_xlabel('Song Title')
    axs[0].set_ylabel('Popularity')
    axs[0].tick_params(axis='x', rotation=90)

    top_5_artists = artists_df[4].astype(int).argsort()[-5:][::-1]
    axs[1].bar(artists_df[3][top_5_artists], artists_df[4][top_5_artists].astype(int), color='purple', width=0.4)
    axs[1].set_title('Top 5 Spotify Artists in Playlist')
    axs[1].set_xlabel('Artist')
    axs[1].set_ylabel('Popularity')
    
    plt.tight_layout()
    plt.show()

def playlist_analyzer():
    access_token = getAuth()
    playlist_info = getPlaylistsid(access_token)
    df_tracks = getNumpyplaylist(playlist_info, access_token)

    unique_artist_ids = np.unique(df_tracks[7])
    df_artists = getNumpyartist(unique_artist_ids, access_token)
 
    unique_track_ids = np.unique(df_tracks[0])
    df_audio_features = getnumpyAudiofeatures(unique_track_ids, access_token)
    plotTracks(df_tracks)
    plotArtists(df_artists)
    plotAudioFeatures(df_audio_features)
    plotTopofPlaylist(df_tracks, df_artists)

# t = time.process_time()
# playlist_analyzer() 
# elapsed_time = time.process_time() - t
# print('Time elapsed:', elapsed_time,'s')


Error compiling Cython file:
------------------------------------------------------------
...
cimport numpy as np
!run packages.py
^
------------------------------------------------------------

/Users/keegankelly/.cache/ipython/cython/_cython_magic_4cd41a1e963108f511195bb31b01db03afb68d69.pyx:2:0: Expected an identifier or literal


In [4]:
%timeit -n 1 playlist_analyzer()

NameError: name 'playlist_analyzer' is not defined